# Project_2 — Original Script (kept) + Problem 1 Add-on (sklearn, Gensim, PySpark)
This notebook preserves the **same structure & variable names** as your original `Project_2` workflow for **Problem 2 (Clustering)**, and then **appends** the full **Problem 1 (Content-based Recommendation)** blocks at the end.


In [ ]:

# (Optional) Install packages if missing
# !pip install gensim pyvi pyspark scikit-learn seaborn matplotlib pandas numpy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score

try:
    from pyvi.ViTokenizer import tokenize as vi_tokenize
except Exception:
    vi_tokenize = None

# Load Vietnamese stopwords (same file name used in your project)
STOP_WORD_FILE = 'vietnamese-stopwords.txt'
try:
    with open(STOP_WORD_FILE, 'r', encoding='utf-8') as f:
        stop_words = [w.strip() for w in f.read().splitlines() if w.strip()]
except FileNotFoundError:
    stop_words = ['và', 'là', 'của', 'những', 'các', 'một', 'như', 'với', 'cho', 'đã', 'đang']
print(f"Stopwords loaded: {len(stop_words)} terms")


In [ ]:

# Load dataset (the original used 'data_motobikes.xlsx')
path_candidates = ['data_motobikes.xlsx', 'data_motorbikes.xlsx']
path = None
for p in path_candidates:
    if os.path.exists(p):
        path = p
        break
if path is None:
    raise FileNotFoundError("Place data_motobikes.xlsx (or data_motorbikes.xlsx) in the working directory.")
print(f"Using: {path}")

df = pd.read_excel(path, engine='openpyxl')
print("Shape trước xử lý:", df.shape)
print(df.info())


In [ ]:

# Xử lý các cột liên quan giá và năm đăng ký

def clean_price(x):
    if isinstance(x, str):
        x = x.replace('.', '').replace(' đ', '').strip()
        try:
            return float(x)
        except:
            return np.nan
    return x

def clean_range(x):
    if isinstance(x, str):
        x = x.replace(' tr', '').replace(',', '.').strip()
        try:
            return float(x) * 1_000_000
        except:
            return np.nan
    return x

def clean_year(x):
    if isinstance(x, str) and 'trước năm 1980' in x.lower():
        return 1980
    try:
        return int(x)
    except:
        return np.nan

if 'Giá' in df.columns:
    df['Giá'] = df['Giá'].apply(clean_price)
if 'Khoảng giá min' in df.columns:
    df['Khoảng giá min'] = df['Khoảng giá min'].apply(clean_range)
if 'Khoảng giá max' in df.columns:
    df['Khoảng giá max'] = df['Khoảng giá max'].apply(clean_range)
if 'Năm đăng ký' in df.columns:
    df['Năm đăng ký'] = df['Năm đăng ký'].apply(clean_year)

# Loại outlier theo IQR cho cột Giá
Q1 = df['Giá'].quantile(0.25)
Q3 = df['Giá'].quantile(0.75)
IQR = Q3 - Q1
lower_bound, upper_bound = Q1 - 1.5*IQR, Q3 + 1.5*IQR

df_clean = df[(df['Giá'] >= lower_bound) & (df['Giá'] <= upper_bound)].copy()

# Điền thiếu
categorical_cols = [c for c in df_clean.columns if df_clean[c].dtype == 'object']
numeric_cols = [c for c in df_clean.columns if pd.api.types.is_numeric_dtype(df_clean[c])]
for c in numeric_cols:
    df_clean[c] = df_clean[c].fillna(df_clean[c].median())
for c in categorical_cols:
    df_clean[c] = df_clean[c].fillna('Không rõ')

print("Sau làm sạch:", df_clean.shape)


In [ ]:

CURRENT_YEAR = 2025
if 'Năm đăng ký' in df_clean.columns:
    df_clean['Tuổi của xe'] = CURRENT_YEAR - df_clean['Năm đăng ký']
else:
    df_clean['Tuổi của xe'] = np.nan

df_fe = df_clean.copy()

# Chuẩn hoá text tương tự bản gốc
text_cols = ['Tiêu đề','Mô tả chi tiết','Thương hiệu','Địa chỉ','Dòng xe']

def normalize_text(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    if vi_tokenize:
        text = vi_tokenize(text)
    text = re.sub(r'\W+', ' ', text)
    text = ' '.join([w for w in text.split() if w not in stop_words])
    return text.strip()

for col in text_cols:
    if col in df_fe.columns:
        df_fe[f'{col}_normalized'] = df_fe[col].apply(normalize_text)

# Kết hợp nội dung
parts = []
for col in ['Tiêu đề_normalized','Mô tả chi tiết_normalized','Thương hiệu_normalized','Địa chỉ_normalized','Dòng xe_normalized']:
    parts.append(df_fe[col] if col in df_fe.columns else '')
df_fe['Text_combined'] = (
    (parts[0] if len(parts)>0 else '') + ' ' +
    (parts[1] if len(parts)>1 else '') + ' ' +
    (parts[2] if len(parts)>2 else '') + ' ' +
    (parts[3] if len(parts)>3 else '') + ' ' +
    (parts[4] if len(parts)>4 else '')
)

# Numeric engineering giống bản gốc
if all(c in df_fe.columns for c in ['Khoảng giá min','Khoảng giá max']):
    df_fe['Giá trung bình'] = (df_fe['Khoảng giá min'] + df_fe['Khoảng giá max'])/2
else:
    df_fe['Giá trung bình'] = np.nan

if 'Giá' in df_fe.columns:
    df_fe['Tỷ lệ giá'] = df_fe['Giá'] / df_fe['Giá trung bình'].replace(0, np.nan)
else:
    df_fe['Tỷ lệ giá'] = np.nan

if 'Số Km đã đi' in df_fe.columns:
    df_fe['Km per year'] = df_fe['Số Km đã đi'] / (df_fe['Tuổi của xe'] + 1e-5)
    df_fe['Số Km đã đi_log'] = np.log1p(df_fe['Số Km đã đi'])
else:
    df_fe['Km per year'] = np.nan
    df_fe['Số Km đã đi_log'] = np.nan

if 'Giá' in df_fe.columns:
    df_fe['Giá_log'] = np.log1p(df_fe['Giá'])
else:
    df_fe['Giá_log'] = np.nan

# LabelEncoder cho một số cột phân loại
for col in ['Thương hiệu','Dòng xe','Loại xe','Dung tích xe','Xuất xứ']:
    if col in df_fe.columns:
        le = LabelEncoder()
        df_fe[f'{col}_label'] = le.fit_transform(df_fe[col].astype(str))

# Chuẩn hoá cột số
num_cols = [c for c in ['Giá','Khoảng giá min','Khoảng giá max','Năm đăng ký','Số Km đã đi','Tuổi của xe','Tỷ lệ giá','Km per year','Giá_log','Số Km đã đi_log'] if c in df_fe.columns]
scaler = StandardScaler()
df_fe[num_cols] = scaler.fit_transform(df_fe[num_cols])

print('Feature set ready:', df_fe.shape)


In [ ]:

# Text vector hoá cho clustering (SVD 100 chiều như trong bản gốc)
text_vect = TfidfVectorizer(max_features=1000, ngram_range=(1,2), analyzer='word', stop_words=stop_words, min_df=2)
X_text = text_vect.fit_transform(df_fe['Text_combined'].fillna(''))
svd = TruncatedSVD(n_components=100, random_state=42)
X_text_r = svd.fit_transform(X_text)

# Kết hợp với nhãn + số
label_cols = [c for c in df_fe.columns if c.endswith('_label')]
num_cols = [c for c in ['Giá','Khoảng giá min','Khoảng giá max','Năm đăng ký','Số Km đã đi','Tuổi của xe','Tỷ lệ giá','Km per year','Giá_log','Số Km đã đi_log'] if c in df_fe.columns]
X_others = df_fe[label_cols + num_cols].values

X = np.hstack([X_text_r, X_others])
X = np.nan_to_num(X)

# Tìm K tối ưu theo silhouette cho KMeans
k_range = range(2, 11)
sil_scores = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    sil_scores.append(silhouette_score(X, km.labels_))
opt_k = k_range[int(np.argmax(sil_scores))]
print('Optimal K (KMeans):', opt_k)

kmeans = KMeans(n_clusters=opt_k, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X)
km_sil = silhouette_score(X, kmeans_labels)
km_db  = davies_bouldin_score(X, kmeans_labels)

# GMM
best_g, best_g_sil = None, -1
for g in k_range:
    gmm = GaussianMixture(n_components=g, random_state=42)
    gmm.fit(X)
    s = silhouette_score(X, gmm.predict(X))
    if s > best_g_sil:
        best_g_sil, best_g = s, g

gmm = GaussianMixture(n_components=best_g, random_state=42)
gmm_labels = gmm.fit_predict(X)
gmm_sil = silhouette_score(X, gmm_labels)
gmm_db  = davies_bouldin_score(X, gmm_labels)

# Agglomerative dùng cùng opt_k
agg = AgglomerativeClustering(n_clusters=opt_k, linkage='ward')
agg_labels = agg.fit_predict(X)
agg_sil = silhouette_score(X, agg_labels)
agg_db  = davies_bouldin_score(X, agg_labels)

print(pd.DataFrame({'Model':['KMeans','GMM','Agglomerative'],
                    'Silhouette':[km_sil, gmm_sil, agg_sil],
                    'Davies-Bouldin':[km_db, gmm_db, agg_db]})
      .sort_values('Silhouette', ascending=False))

# Gắn nhãn tốt nhất để profiling (KMeans)
df_fe['cluster_kmeans'] = kmeans_labels
profile = (df_fe.groupby('cluster_kmeans')
           .agg({'Giá':'mean', 'Số Km đã đi':'mean', 'Tuổi của xe':'mean', 'Thương hiệu':lambda x: x.mode()[0] if len(x)>0 else 'NA', 'Loại xe':lambda x: x.mode()[0] if len(x)>0 else 'NA'})
           .rename(columns={'Giá':'Giá TB','Số Km đã đi':'Km TB','Tuổi của xe':'Tuổi TB'}))
print(profile)


In [ ]:

# PySpark clustering theo bản gốc
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, Normalizer, VectorAssembler
from pyspark.ml.clustering import KMeans as SKMeans, GaussianMixture as SGMM, BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import col, monotonically_increasing_id

spark = SparkSession.builder.appName("MarketSegmentation").getOrCreate()

# Build Spark text TF-IDF
sdf = spark.createDataFrame(df_fe[['id','Text_combined']].copy())
_tok = Tokenizer(inputCol='Text_combined', outputCol='tokens')
sdf = _tok.transform(sdf)
_rem = StopWordsRemover(inputCol='tokens', outputCol='filtered', stopWords=stop_words)
sdf = _rem.transform(sdf)
_htf = HashingTF(inputCol='filtered', outputCol='rawFeatures', numFeatures=1<<14)
sdf = _htf.transform(sdf)
_idf = IDF(inputCol='rawFeatures', outputCol='features')
idf_model = _idf.fit(sdf)
sdf = idf_model.transform(sdf)
_norm = Normalizer(inputCol='features', outputCol='normFeatures')
sdf = _norm.transform(sdf)

# Assemble with numeric/label columns
extra_cols = []
for c in ['Giá','Khoảng giá min','Khoảng giá max','Năm đăng ký','Số Km đã đi','Tuổi của xe','Tỷ lệ giá','Km per year','Giá_log','Số Km đã đi_log',
          'Thương hiệu_label','Dòng xe_label','Loại xe_label','Dung tích xe_label','Xuất xứ_label']:
    if c in df_fe.columns:
        extra_cols.append(c)

sdf_extra = spark.createDataFrame(df_fe[extra_cols])
sdf_feat = sdf.select('normFeatures').withColumnRenamed('normFeatures','textFeat')

sdf_feat = sdf_feat.withColumn('row_id', monotonically_increasing_id())
sdf_extra = sdf_extra.withColumn('row_id', monotonically_increasing_id())
sdf_join = sdf_feat.join(sdf_extra, on='row_id').drop('row_id')

assembler = VectorAssembler(inputCols=['textFeat'] + [c for c in sdf_extra.columns if c!='row_id'], outputCol='features')
sdf_vec = assembler.transform(sdf_join)

evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='features')

# KMeans
scores = []
for k in range(2,11):
    m = SKMeans(k=k, seed=42).fit(sdf_vec)
    p = m.transform(sdf_vec)
    scores.append((k, evaluator.evaluate(p)))
opt_k = max(scores, key=lambda x: x[1])[0]
km_model = SKMeans(k=opt_k, seed=42).fit(sdf_vec)
preds_km = km_model.transform(sdf_vec)
km_sil = evaluator.evaluate(preds_km)

# BisectingKMeans
scores_b = []
for k in range(2,11):
    m = BisectingKMeans(k=k, seed=42).fit(sdf_vec)
    p = m.transform(sdf_vec)
    scores_b.append((k, evaluator.evaluate(p)))
opt_b = max(scores_b, key=lambda x: x[1])[0]
bkm_model = BisectingKMeans(k=opt_b, seed=42).fit(sdf_vec)
preds_b = bkm_model.transform(sdf_vec)
bkm_sil = evaluator.evaluate(preds_b)

# GMM (có thể silhouette âm với dữ liệu sparse)
try:
    gmm_model = SGMM(k=2, seed=42).fit(sdf_vec)
    preds_g = gmm_model.transform(sdf_vec)
    gmm_sil = evaluator.evaluate(preds_g)
except Exception:
    gmm_sil = float('nan')

print(pd.DataFrame({'Model':['KMeans','BisectingKMeans','GMM'], 'Silhouette':[km_sil, bkm_sil, gmm_sil]}))



---

# Problem 1 — Content-Based Recommendation (Sklearn + Gensim + PySpark)
**Lưu ý:** Các cell dưới đây **không thay đổi** code gốc, chỉ **bổ sung** hàm gợi ý theo yêu cầu Topic (cosine_similarity & gensim) và phiên bản PySpark.


In [ ]:

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

assert 'df_fe' in globals(), "df_fe chưa sẵn sàng. Hãy chạy phần gốc trước."
assert 'Text_combined' in df_fe.columns, "Thiếu cột Text_combined từ phần Feature Engineering."

# Heuristic chọn cấu hình tốt nhất: brand-consistency
def brand_consistency(sim_matrix, brands, topk=10):
    n = sim_matrix.shape[0]
    if brands is None:
        return 0.0
    same, total = 0, 0
    for i in range(n):
        sims = sim_matrix[i]
        idx = np.argpartition(-sims, range(1, topk+1))[1:topk+1]
        base = brands[i]
        total += len(idx)
        same += sum(1 for j in idx if brands[j] == base)
    return same / max(total, 1)

texts = df_fe['Text_combined'].fillna('')
brands = df_fe['Thương hiệu'] if 'Thương hiệu' in df_fe.columns else None

max_features_grid = [3000, 5000, 8000]
ngram_grid = [(1,1), (1,2)]
svd_components = 200

best_score = -1
best_cfg = None
best_sim = None
best_vectorizer = None
best_svd = None

for mf in max_features_grid:
    for ng in ngram_grid:
        vect = TfidfVectorizer(max_features=mf, ngram_range=ng, analyzer='word', stop_words=stop_words, min_df=2)
        X = vect.fit_transform(texts)
        if X.shape[1] > svd_components:
            svd = TruncatedSVD(n_components=svd_components, random_state=42)
            Xr = svd.fit_transform(X)
            sim = cosine_similarity(Xr)
        else:
            svd = None
            sim = cosine_similarity(X)
        score = brand_consistency(sim, brands)
        if score > best_score:
            best_score = score
            best_cfg = (mf, ng)
            best_sim = sim
            best_vectorizer = vect
            best_svd = svd

print(f"[Problem1][sklearn] Best TF-IDF: max_features={best_cfg[0]}, ngram={best_cfg[1]}, brand-consistency={best_score:.3f}")

# Helper để recommend theo id

def _get_index_by_id(df, item_id, id_col='id'):
    idx = df.index[df[id_col] == item_id].tolist()
    if not idx:
        raise ValueError(f"Item id {item_id} không tồn tại ở cột '{id_col}'")
    return idx[0]

def recommend_by_id_sklearn(df_source, sim_matrix, item_id, topn=5, id_col='id', cols=('id','Tiêu đề','Thương hiệu','Giá')):
    i = _get_index_by_id(df_source, item_id, id_col=id_col)
    sims = sim_matrix[i].copy()
    order = np.argsort(-sims)
    order = [j for j in order if j != i][:topn]
    out = df_source.iloc[order].copy()
    out['similarity'] = sims[order]
    keep_cols = [c for c in cols if c in out.columns] + ['similarity']
    return out[keep_cols]


In [ ]:

from gensim import corpora, models, similarities

texts = [str(x) for x in df_fe['Text_combined'].fillna('')]
content_tokens = [t.split() for t in texts]
content_tokens = [[w for w in doc if w not in stop_words] for doc in content_tokens]

dictionary = corpora.Dictionary(content_tokens)
corpus = [dictionary.doc2bow(doc) for doc in content_tokens]
model_tfidf = models.TfidfModel(corpus)
index = similarities.SparseMatrixSimilarity(model_tfidf[corpus], num_features=len(dictionary))


def recommend_by_id_gensim(df_source, item_id, topn=5, id_col='id', cols=('id','Tiêu đề','Thương hiệu','Giá')):
    i = df_source.index[df_source[id_col] == item_id].tolist()
    if not i:
        raise ValueError(f"Item id {item_id} không tồn tại")
    i = i[0]
    vec = dictionary.doc2bow(content_tokens[i])
    sims = index[model_tfidf[vec]]
    order = np.argsort(-sims)
    order = [j for j in order if j != i][:topn]
    out = df_source.iloc[order].copy()
    out['similarity'] = sims[order]
    keep_cols = [c for c in cols if c in out.columns] + ['similarity']
    return out[keep_cols]


In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, Normalizer
from pyspark.ml.feature import BucketedRandomProjectionLSH

spark = SparkSession.builder.appName("Problem1_ContentBased").getOrCreate()

use_cols = ['id','Tiêu đề','Thương hiệu','Giá','Text_combined']
use_cols = [c for c in use_cols if c in df_fe.columns]
sdf_cb = spark.createDataFrame(df_fe[use_cols])

_tok2 = Tokenizer(inputCol='Text_combined', outputCol='tokens')
sdf_cb = _tok2.transform(sdf_cb)
_rem2 = StopWordsRemover(inputCol='tokens', outputCol='filtered', stopWords=stop_words)
sdf_cb = _rem2.transform(sdf_cb)
_htf2 = HashingTF(inputCol='filtered', outputCol='rawFeatures', numFeatures=1<<14)
sdf_cb = _htf2.transform(sdf_cb)
_idf2 = IDF(inputCol='rawFeatures', outputCol='features')
idf_model2 = _idf2.fit(sdf_cb)
sdf_cb = idf_model2.transform(sdf_cb)
_norm2 = Normalizer(inputCol='features', outputCol='normFeatures')
sdf_cb = _norm2.transform(sdf_cb)

_lsh2 = BucketedRandomProjectionLSH(inputCol='normFeatures', outputCol='hashes', bucketLength=2.0, numHashTables=5)
lsh_model2 = _lsh2.fit(sdf_cb)


def recommend_by_id_spark(item_id, topn=5):
    item = sdf_cb.filter(col('id') == item_id).limit(1)
    if item.count() == 0:
        raise ValueError(f"Item id {item_id} không tồn tại")
    res = lsh_model2.approxNearestNeighbors(sdf_cb, item.collect()[0]['normFeatures'], topn+1)
    res = res.filter(col('id') != item_id).orderBy(col('distCol').asc())
    return res.select('id','Tiêu đề','Thương hiệu','Giá','distCol').toPandas()


### Usage
```python
# Ví dụ: lấy id đầu tiên
item_id = int(df_fe['id'].iloc[0])

# Sklearn (cosine)
recommend_by_id_sklearn(df_fe, best_sim, item_id, topn=5)

# Gensim
recommend_by_id_gensim(df_fe, item_id, topn=5)

# PySpark
recommend_by_id_spark(item_id, topn=5)
```
